In [1]:
#import needed libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, AveragePooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.cross_validation import train_test_split


Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#read the input dataset
df = pd.read_csv("./train.csv")
#split the data into train and test sets
train, test = train_test_split(df, test_size=0.2)

#Replace all blank comments with text in training set
#extract training comments 
comments_train = train["comment_text"].fillna("cbarcelon").values
#extract the toxciity ratings
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_ratings = train[classes].values
test_ratings = test[classes].values
#extract test comments
comments_test = test["comment_text"].fillna("cbarcelon").values

#tokenizer the text
#vectorize text
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(list(comments_train))
tokenized_comments_train = tokenizer.texts_to_sequences(comments_train)
tokenized_comments_test = tokenizer.texts_to_sequences(comments_test)
#pad the text so each comment is uniform in length
X_train = sequence.pad_sequences(tokenized_comments_train, maxlen= 50, truncating='post')
X_test = sequence.pad_sequences(tokenized_comments_test, maxlen=50,  truncating='post')

In [43]:
#define LSTM sequential model
lstm = Sequential()
lstm.add(Embedding(5000, output_dim=128))
lstm.add(Bidirectional(LSTM(50, return_sequences=True), merge_mode='ave'))
lstm.add(Bidirectional(LSTM(50, return_sequences=True), merge_mode='ave'))
lstm.add(Bidirectional(LSTM(50, return_sequences=True), merge_mode='ave'))
lstm.add(GlobalMaxPool1D())
lstm.add(Dropout(.5))
lstm.add(Dense(100, activation='relu'))
lstm.add(Dropout(.5))
lstm.add(Dense(6, activation = "sigmoid"))

lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 128)         640000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 50)          71600     
_________________________________________________________________
bidirectional_5 (Bidirection (None, None, 50)          40400     
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 50)          40400     
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 50)                0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 100)               5100      
__________

In [44]:
#compile the model
lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#create checkpoint file
file_path = "weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#early stop checkpoint
early = EarlyStopping(monitor='val_loss', mode='min', patience=20)
callbacks_list = [checkpoint, early] 

In [45]:
#train/fit the model
lstm.fit(X_train, train_ratings, batch_size=750, epochs=10, validation_split=0.1, callbacks=callbacks_list)

Train on 69012 samples, validate on 7668 samples
Epoch 1/10
69012/69012 [==============================] - 53s - loss: 0.2565 - acc: 0.9434 - val_loss: 0.1462 - val_acc: 0.9620
Epoch 2/10
69012/69012 [==============================] - 51s - loss: 0.1552 - acc: 0.9633 - val_loss: 0.1455 - val_acc: 0.9620
Epoch 3/10
69012/69012 [==============================] - 51s - loss: 0.1270 - acc: 0.9645 - val_loss: 0.0877 - val_acc: 0.9732
Epoch 4/10
69012/69012 [==============================] - 51s - loss: 0.0832 - acc: 0.9744 - val_loss: 0.0733 - val_acc: 0.9760
Epoch 5/10
69012/69012 [==============================] - 51s - loss: 0.0697 - acc: 0.9778 - val_loss: 0.0678 - val_acc: 0.9776
Epoch 6/10
69012/69012 [==============================] - 51s - loss: 0.0643 - acc: 0.9790 - val_loss: 0.0655 - val_acc: 0.9785
Epoch 7/10
69012/69012 [==============================] - 51s - loss: 0.0596 - acc: 0.9801 - val_loss: 0.0676 - val_acc: 0.9783
Epoch 8/10
69012/69012 [==============================]

In [46]:
#load the best weights
lstm.load_weights(file_path)

#make predictions on test set
pred = lstm.predict(X_test)

In [47]:
#log loss score function
from sklearn.metrics import log_loss
def calc_loss(y_true, y_pred):
    return np.mean([log_loss(y_true[:, i], y_pred[:, i]) 
                    for i in range(y_true.shape[1])])

In [48]:
score = calc_loss(test_ratings, pred)
print(score)

0.065692556162
